In [26]:
import numpy as np
from sklearn import tree,metrics,svm,ensemble,neighbors
import pandas as pd
from sklearn.model_selection import train_test_split,KFold
from sklearn import linear_model
import xgboost as xgb
from sklearn.preprocessing import PolynomialFeatures
from sklearn.pipeline import Pipeline

In [35]:
mixture = ['氮化钒铁FeV55N11-A','低铝硅铁','钒氮合金(进口)','钒铁(FeV50-A)','钒铁(FeV50-B)','硅铝钙','硅铝合金FeAl30Si25','硅铝锰合金球',
           '硅锰面（硅锰渣）','硅铁(合格块)','硅铁FeSi75-B','石油焦增碳剂','锰硅合金FeMn64Si27(合格块)','锰硅合金FeMn68Si18(合格块)',
           '碳化硅(55%)','硅钙碳脱氧剂']
columns = ['转炉终点温度', '转炉终点C', '转炉终点Mn', '转炉终点S', '转炉终点P',
       '转炉终点Si', '钢水净重', '连铸正样C', '连铸正样Mn', '连铸正样S', '连铸正样P', '连铸正样Si', '氮化钒铁FeV55N11-A', '低铝硅铁',
       '钒氮合金(进口)', '钒铁(FeV50-A)', '钒铁(FeV50-B)', '钒铁(FeV50-B).1', '硅铝钙',
       '硅铝合金FeAl30Si25', '硅铝锰合金球', '硅锰面（硅锰渣）', '硅铁(合格块)', '硅铁FeSi75-B',
       '石油焦增碳剂', '锰硅合金FeMn64Si27(合格块)', '锰硅合金FeMn68Si18(合格块)', '碳化硅(55%)',
       '硅钙碳脱氧剂']
x_col = ['转炉终点温度', '转炉终点C', '转炉终点Mn', '转炉终点S', '转炉终点P',
       '转炉终点Si', '钢水净重', '氮化钒铁FeV55N11-A', '低铝硅铁',
       '钒氮合金(进口)', '钒铁(FeV50-A)', '钒铁(FeV50-B)', '钒铁(FeV50-B).1', '硅铝钙',
       '硅铝合金FeAl30Si25', '硅铝锰合金球', '硅锰面（硅锰渣）', '硅铁(合格块)', '硅铁FeSi75-B',
       '石油焦增碳剂', '锰硅合金FeMn64Si27(合格块)', '锰硅合金FeMn68Si18(合格块)', '碳化硅(55%)',
       '硅钙碳脱氧剂']
price = [350000,6500,350000,205000,205000,11800,1000,8500,7600,6000,6000,4600,8150,8150,6100,4000]
len(price)

16

In [66]:
data = pd.read_excel('./data1.xlsx')[columns]
data.drop(data[data['连铸正样C'].isnull()].index,inplace=True)
data.fillna(data.mean(),inplace=True)
# X = data[x_col].apply(lambda x: (x - np.min(x)) / (np.max(x) - np.min(x)+1e-10)).values
X = data[x_col].values
data2 = pd.read_excel('data2.xlsx')
data2.head(2)
len(data)#.isna().sum()

810

In [67]:
# data[x_col].apply(lambda x: (x - np.min(x)) / (np.max(x) - np.min(x)+1e-10))

In [95]:
Y = data['连铸正样Mn'].values
clf = linear_model.BayesianRidge()
ki = 8
kf = KFold(n_splits=ki)
r2,mae,mse=0,0,0
for train_index,test_index in kf.split(X):
    clf_Mn = clf.fit(X[train_index],Y[train_index])
    pre_y = clf_Mn.predict(X[test_index])
    r2+=metrics.r2_score(Y[test_index],pre_y)
    mae+=metrics.mean_absolute_error(Y[test_index],pre_y)
    mse+=metrics.mean_squared_error(Y[test_index],pre_y)
r2/ki,mae/ki,mse/ki,np.mean(np.abs((Y[test_index]-pre_y)/Y[test_index]))

(0.12182281488734231,
 0.0002577557637286876,
 1.0833932251544438e-07,
 0.025244462996490847)

In [96]:
Y = data['连铸正样C'].values
clf = linear_model.LinearRegression()
ki = 8
kf = KFold(n_splits=ki)
r2,mae,mse=0,0,0
for train_index,test_index in kf.split(X):
    clf_C = clf.fit(X[train_index],Y[train_index])
    pre_y = clf_C.predict(X[test_index])
    r2+=metrics.r2_score(Y[test_index],pre_y)
    mae+=metrics.mean_absolute_error(Y[test_index],pre_y)
    mse+=metrics.mean_squared_error(Y[test_index],pre_y)
r2/ki,mae/ki,mse/ki,np.mean(np.abs((Y[test_index]-pre_y)/Y[test_index]))

(-0.17835828857335728,
 7.18033521100905e-05,
 1.0677563633536286e-08,
 0.02528586954827705)

In [97]:
Y = data['连铸正样S'].values
clf = linear_model.LinearRegression()
ki = 8
kf = KFold(n_splits=ki)
r2,mae,mse=0,0,0
for train_index,test_index in kf.split(X):
    clf_S = clf.fit(X[train_index],Y[train_index])
    pre_y = clf_S.predict(X[test_index])
    r2+=metrics.r2_score(Y[test_index],pre_y)
    mae+=metrics.mean_absolute_error(Y[test_index],pre_y)
    mse+=metrics.mean_squared_error(Y[test_index],pre_y)
r2/ki,mae/ki,mse/ki,np.mean(np.abs((Y[test_index]-pre_y)/Y[test_index]))

(0.0859079053308286,
 2.551226723204416e-05,
 1.0739373645547263e-09,
 0.06865947998699251)

In [102]:
Y = data['连铸正样P'].values
clf = linear_model.BayesianRidge()
ki = 8
kf = KFold(n_splits=ki)
r2,mae,mse=0,0,0
for train_index,test_index in kf.split(X):
    clf_P = clf.fit(X[train_index],Y[train_index])
    pre_y = clf_P.predict(X[test_index])
    r2+=metrics.r2_score(Y[test_index],pre_y)
    mae+=metrics.mean_absolute_error(Y[test_index],pre_y)
    mse+=metrics.mean_squared_error(Y[test_index],pre_y)
r2/ki,mae/ki,mse/ki,np.mean(np.abs((Y[test_index]-pre_y)/Y[test_index]))

(-0.09013455540329227,
 4.5149665120858574e-05,
 3.0365705808088065e-09,
 0.2119461422450102)

In [100]:
Y = data['连铸正样Si'].values
clf = linear_model.LinearRegression()
ki = 8
kf = KFold(n_splits=ki)
r2,mae,mse=0,0,0
for train_index,test_index in kf.split(X):
    clf_Mn = clf.fit(X[train_index],Y[train_index])
    pre_y = clf_Mn.predict(X[test_index])
    r2+=metrics.r2_score(Y[test_index],pre_y)
    mae+=metrics.mean_absolute_error(Y[test_index],pre_y)
    mse+=metrics.mean_squared_error(Y[test_index],pre_y)
r2/ki,mae/ki,mse/ki,np.mean(np.abs((Y[test_index]-pre_y)/Y[test_index]))

(0.11405754250228836,
 0.00016228735185103664,
 4.8928400195975566e-08,
 0.04444014475604282)

In [79]:
init_C,init_Mn,init_P,init_S,init_Si = data['转炉终点C'].mean(),data['转炉终点Mn'].mean(),data['转炉终点P'].mean(),data['转炉终点S'].mean(),data['转炉终点Si'].mean()

In [87]:
clf_C.coef_

array([ 2.90478395e-08,  9.05874192e-03, -1.70139773e-02,  2.24288135e-02,
       -1.71989719e-01,  2.56243437e-05,  9.12487282e-10,  1.99837767e-06,
       -9.09143992e-07,  1.59568232e-06,  7.32502551e-07,  2.33721701e-12,
        1.86676623e-06, -6.30146473e-07, -1.89737647e-06,  9.00546998e-14,
       -2.44747705e-10,  6.45923412e-07,  3.19284574e-07,  1.15735708e-06,
        2.80408420e-08,  3.86359678e-09,  1.53718655e-06,  1.20559566e-06])